In [2]:
import os
import sys
from collections import defaultdict
from functools import reduce, partial

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import skbio
from skbio import diversity
from skbio.stats import ordination
from sklearn.manifold import MDS

In [ ]:
otus = pd.read_csv("../data/otus.csv")
comp = pd.read_csv("../data/comp.csv")

In [3]:
# beta_diversity, get_beta_diversity_metrics
# cca
# MDS(2, metric=False)

In [4]:
diversity.get_beta_diversity_metrics()

['unweighted_unifrac', 'weighted_unifrac']

In [6]:
diversity.beta_diversity?

Signature:
diversity.beta_diversity(
    metric,
    counts,
    ids=None,
    validate=True,
    pairwise_func=None,
    **kwargs,
)
Docstring:
Compute distances between all pairs of samples

State: Experimental as of 0.4.0.

Parameters
----------
metric : str, callable
    The pairwise distance function to apply. See the scipy ``pdist`` docs
    and the scikit-bio functions linked under *See Also* for available
    metrics. Passing metrics as a strings is preferable as this often
    results in an optimized version of the metric being used.
counts : 2D array_like of ints or floats or 2D pandas DataFrame
    Matrix containing count/abundance data where each row contains counts
    of OTUs in a given sample.
ids : iterable of strs, optional
    Identifiers for each sample in ``counts``. By default, samples will be
    assigned integer identifiers in the order that they were provided
    (where the type of the identifiers will be ``str``).
validate : bool, optional
    If `False`, valid

In [7]:
ordination.cca?

Signature: ordination.cca(y, x, scaling=1)
Docstring:
Compute canonical (also known as constrained) correspondence

State: Experimental as of 0.4.0.
analysis.

Canonical (or constrained) correspondence analysis is a
multivariate ordination technique. It appeared in community
ecology [1]_ and relates community composition to the variation in
the environment (or in other factors). It works from data on
abundances or counts of samples and constraints variables,
and outputs ordination axes that maximize sample separation among species.

It is better suited to extract the niches of taxa than linear
multivariate methods because it assumes unimodal response curves
(habitat preferences are often unimodal functions of habitat
variables [2]_).

As more environmental variables are added, the result gets more
similar to unconstrained ordination, so only the variables that
are deemed explanatory should be included in the analysis.

Parameters
----------
y : DataFrame
    Samples by features table (

In [8]:
MDS?

Init signature:
MDS(
    n_components=2,
    *,
    metric=True,
    n_init=4,
    max_iter=300,
    verbose=0,
    eps=0.001,
    n_jobs=None,
    random_state=None,
    dissimilarity='euclidean',
)
Docstring:     
Multidimensional scaling.

Read more in the :ref:`User Guide <multidimensional_scaling>`.

Parameters
----------
n_components : int, default=2
    Number of dimensions in which to immerse the dissimilarities.

metric : bool, default=True
    If ``True``, perform metric MDS; otherwise, perform nonmetric MDS.

n_init : int, default=4
    Number of times the SMACOF algorithm will be run with different
    initializations. The final results will be the best output of the runs,
    determined by the run with the smallest final stress.

max_iter : int, default=300
    Maximum number of iterations of the SMACOF algorithm for a single run.

verbose : int, default=0
    Level of verbosity.

eps : float, default=1e-3
    Relative tolerance with respect to stress at which to declare
 